# records

In [1]:
import vectorbt as vbt

In [2]:
import numpy as np
import pandas as pd
from numba import njit
from collections import namedtuple
from datetime import datetime

In [3]:
# Disable caching for performance testing
# NOTE: Expect waterfall of executions, since some attributes depend on other attributes 
# that have to be calculated again and again
vbt.settings.caching['enabled'] = False

In [4]:
example_dt = np.dtype([
    ('id', np.int64),
    ('idx', np.int64),
    ('col', np.int64),
    ('some_field1', np.float64),
    ('some_field2', np.float64)
], align=True)

In [5]:
records_arr = np.asarray([
    (0, 0, 0, 10, 21),
    (1, 1, 0, 11, 20),
    (2, 2, 0, 12, 19),
    (3, 0, 1, 13, 18),
    (4, 1, 1, 14, 17),
    (5, 2, 1, 13, 18),
    (6, 0, 2, 12, 19),
    (7, 1, 2, 11, 20),
    (8, 2, 2, 10, 21)
], dtype=example_dt)
print(records_arr)
print(records_arr.shape)

columns = pd.MultiIndex.from_arrays([[0, 1, 1, 1], ['a', 'b', 'c', 'd']], names=['lvl1', 'lvl2'])
wrapper = vbt.ArrayWrapper(index=['x', 'y', 'z'], columns=columns, ndim=2, freq='1 days')
records = vbt.Records(wrapper, records_arr)

records_grouped = vbt.Records(wrapper.copy(group_by=0), records_arr)

In [6]:
big_records_arr = np.asarray(list(zip(*(
    np.arange(1000000),
    np.tile(np.arange(1000), 1000),
    np.repeat(np.arange(1000), 1000),
    np.random.randint(0, 100, size=1000000),
    np.random.randint(0, 100, size=1000000)))), dtype=example_dt)
print(big_records_arr.shape)

big_columns = pd.MultiIndex.from_arrays([np.repeat(np.array([0, 1]), 500), np.arange(1000)], names=['lvl1', 'lvl2'])
big_wrapper = vbt.ArrayWrapper(index=np.arange(1000), columns=big_columns, ndim=2, freq='1 days')
big_records = vbt.Records(big_wrapper, big_records_arr)

big_records_grouped = vbt.Records(big_wrapper.copy(group_by=0), big_records_arr)

In [7]:
records_nosort = records.copy(records_arr=records.records_arr[::-1])
print(records_nosort.records_arr)

big_records_nosort = big_records.copy(records_arr=big_records.records_arr[::-1])

In [8]:
group_by = pd.Series(['first', 'first', 'second', 'second'], name='group')
big_group_by = pd.Series(np.repeat(np.array([0, 1]), 500))

## ColumnMapper

In [9]:
print(records.col_mapper.col_arr)

In [10]:
print(records.col_mapper.get_col_arr())
print(records_grouped.col_mapper.get_col_arr())

%timeit big_records_grouped.col_mapper.get_col_arr()

In [11]:
print(records.col_mapper.col_range)

%timeit big_records.col_mapper.col_range

In [12]:
print(records.col_mapper.get_col_range())
print(records_grouped.col_mapper.get_col_range())

%timeit big_records_grouped.col_mapper.get_col_range()

In [13]:
print(records.col_mapper.col_map)

%timeit big_records.col_mapper.col_map

In [14]:
print(records.col_mapper.get_col_map())
print(records_grouped.col_mapper.get_col_map())

%timeit big_records_grouped.col_mapper.get_col_map()

In [15]:
print(records.col_mapper.is_sorted())
%timeit big_records.col_mapper.is_sorted()

print(records_nosort.col_mapper.is_sorted())
%timeit big_records_nosort.col_mapper.is_sorted()

## MappedArray

In [9]:
mapped_array = records.map_field('some_field1')
big_mapped_array = big_records.map_field('some_field1')

mapped_array_nosort = records_nosort.map_field('some_field1')
big_mapped_array_nosort = big_records_nosort.map_field('some_field1')

mapped_array_grouped = records_grouped.map_field('some_field1')
big_mapped_array_grouped = big_records_grouped.map_field('some_field1')

In [12]:
print(mapped_array[(0, 'a')].values)
print(mapped_array[(0, 'a')].col_arr)
print(mapped_array[(0, 'a')].wrapper.columns)

print(mapped_array[(1, 'b')].values)
print(mapped_array[(1, 'b')].col_arr)
print(mapped_array[(1, 'b')].wrapper.columns)

print(mapped_array[[(0, 'a'), (0, 'a')]].values)
print(mapped_array[[(0, 'a'), (0, 'a')]].col_arr)
print(mapped_array[[(0, 'a'), (0, 'a')]].wrapper.columns)

print(mapped_array[[(0, 'a'), (1, 'b')]].values)
print(mapped_array[[(0, 'a'), (1, 'b')]].col_arr)
print(mapped_array[[(0, 'a'), (1, 'b')]].wrapper.columns)

%timeit big_mapped_array.iloc[0]
%timeit big_mapped_array.iloc[:]

In [13]:
print(mapped_array_nosort[(0, 'a')].values)
print(mapped_array_nosort[(0, 'a')].col_arr)
print(mapped_array_nosort[(0, 'a')].wrapper.columns)

print(mapped_array_nosort[(1, 'b')].values)
print(mapped_array_nosort[(1, 'b')].col_arr)
print(mapped_array_nosort[(1, 'b')].wrapper.columns)

print(mapped_array_nosort[[(0, 'a'), (0, 'a')]].values)
print(mapped_array_nosort[[(0, 'a'), (0, 'a')]].col_arr)
print(mapped_array_nosort[[(0, 'a'), (0, 'a')]].wrapper.columns)

print(mapped_array_nosort[[(0, 'a'), (1, 'b')]].values)
print(mapped_array_nosort[[(0, 'a'), (1, 'b')]].col_arr)
print(mapped_array_nosort[[(0, 'a'), (1, 'b')]].wrapper.columns)

%timeit big_mapped_array_nosort.iloc[0]
%timeit big_mapped_array_nosort.iloc[:]

In [14]:
print(mapped_array_grouped[0].wrapper.columns)  # indexing on groups, not columns!
print(mapped_array_grouped[0].wrapper.ndim)
print(mapped_array_grouped[0].wrapper.grouped_ndim)
print(mapped_array_grouped[0].wrapper.grouper.group_by)

print(mapped_array_grouped[1].wrapper.columns)
print(mapped_array_grouped[1].wrapper.ndim)
print(mapped_array_grouped[1].wrapper.grouped_ndim)
print(mapped_array_grouped[1].wrapper.grouper.group_by)

print(mapped_array_grouped[[0]].wrapper.columns)
print(mapped_array_grouped[[0]].wrapper.ndim)
print(mapped_array_grouped[[0]].wrapper.grouped_ndim)
print(mapped_array_grouped[[0]].wrapper.grouper.group_by)

print(mapped_array_grouped[[0, 1]].wrapper.columns)
print(mapped_array_grouped[[0, 1]].wrapper.ndim)
print(mapped_array_grouped[[0, 1]].wrapper.grouped_ndim)
print(mapped_array_grouped[[0, 1]].wrapper.grouper.group_by)

%timeit big_mapped_array_grouped.iloc[0]
%timeit big_mapped_array_grouped.iloc[:]

In [15]:
print(mapped_array.wrapper.index)
print(mapped_array.wrapper.columns)
print(mapped_array.wrapper.ndim)
print(mapped_array.wrapper.grouper.group_by)

print(mapped_array_grouped.wrapper.index)
print(mapped_array_grouped.wrapper.columns)
print(mapped_array_grouped.wrapper.ndim)
print(mapped_array_grouped.wrapper.grouper.group_by)

In [16]:
print(mapped_array.values)

In [17]:
print(mapped_array.col_arr)

In [18]:
print(mapped_array.id_arr)

In [19]:
print(mapped_array.idx_arr)

In [20]:
print(mapped_array.is_sorted())
%timeit big_mapped_array.is_sorted()

print(mapped_array_nosort.is_sorted())
%timeit big_mapped_array_nosort.is_sorted()

print(mapped_array.is_sorted(incl_id=True))
%timeit big_mapped_array.is_sorted(incl_id=True)

print(mapped_array_nosort.is_sorted(incl_id=True))
%timeit big_mapped_array_nosort.is_sorted(incl_id=True)

In [21]:
print(mapped_array.sort().col_arr)
print(mapped_array.sort().id_arr)
%timeit big_mapped_array.sort()

print(mapped_array_nosort.sort().col_arr)
print(mapped_array_nosort.sort().id_arr)
%timeit big_mapped_array_nosort.sort()

print(mapped_array.sort(incl_id=True).col_arr)
print(mapped_array.sort(incl_id=True).id_arr)
%timeit big_mapped_array.sort(incl_id=True)

print(mapped_array_nosort.sort(incl_id=True).col_arr)
print(mapped_array_nosort.sort(incl_id=True).id_arr)
%timeit big_mapped_array_nosort.sort(incl_id=True)

In [10]:
mask = mapped_array.values >= mapped_array.values.mean()
print(mapped_array.apply_mask(mask).values)

big_mask = big_mapped_array.values >= big_mapped_array.values.mean()
%timeit big_mapped_array.apply_mask(big_mask)
%timeit big_mapped_array_nosort.apply_mask(big_mask)

In [24]:
@njit
def every_2_nb(inout, idxs, col, mapped_arr):
    inout[idxs[::2]] = True

print(mapped_array.map_to_mask(every_2_nb))
%timeit big_mapped_array.map_to_mask(every_2_nb)
%timeit big_mapped_array_nosort.map_to_mask(every_2_nb)

In [25]:
print(mapped_array.values)

print(mapped_array.top_n_mask(1))
%timeit big_mapped_array.top_n_mask(100)

print(mapped_array.bottom_n_mask(1))
%timeit big_mapped_array.bottom_n_mask(100)

In [26]:
print(mapped_array.top_n(1).values)
%timeit big_mapped_array.top_n(100)

print(mapped_array.bottom_n(1).values)
%timeit big_mapped_array.bottom_n(100)

In [27]:
print(mapped_array.is_expandable())

%timeit big_mapped_array.is_expandable()

In [28]:
print(mapped_array.to_pd())
print(mapped_array.to_pd(fill_value=0.))

%timeit big_mapped_array.to_pd()

In [29]:
print(mapped_array[(0, 'a')].to_pd(ignore_index=True))
%timeit big_mapped_array[0].to_pd(ignore_index=True)

print(mapped_array.to_pd(ignore_index=True))
%timeit big_mapped_array.to_pd(ignore_index=True)

print(mapped_array_grouped.to_pd(ignore_index=True))
print(mapped_array_grouped.to_pd(ignore_index=True, fill_value=0))
%timeit big_mapped_array_grouped.to_pd(ignore_index=True)

In [30]:
@njit
def mean_reduce_nb(col, a):
    return np.mean(a)

print(mapped_array[(0, 'a')].reduce(mean_reduce_nb))
print(mapped_array[[(0, 'a'), (1, 'b')]].reduce(mean_reduce_nb))
print(mapped_array.reduce(mean_reduce_nb))
print(mapped_array.reduce(mean_reduce_nb, fill_value=0.))
print(mapped_array.reduce(mean_reduce_nb, fill_value=0., wrap_kwargs=dict(dtype=np.int_)))
print(mapped_array.reduce(mean_reduce_nb, wrap_kwargs=dict(to_timedelta=True)))

%timeit big_mapped_array.reduce(mean_reduce_nb)
%timeit big_mapped_array_nosort.reduce(mean_reduce_nb)

In [31]:
print(mapped_array_grouped[0].reduce(mean_reduce_nb))
print(mapped_array_grouped[[0]].reduce(mean_reduce_nb))
print(mapped_array_grouped.reduce(mean_reduce_nb))
print(mapped_array_grouped.reduce(mean_reduce_nb, group_by=False))

%timeit big_mapped_array_grouped.reduce(mean_reduce_nb)

In [32]:
@njit
def argmin_reduce_nb(col, a):
    return np.argmin(a)

print(mapped_array.reduce(argmin_reduce_nb, returns_idx=True))
%timeit big_mapped_array.reduce(argmin_reduce_nb, returns_idx=True)

print(mapped_array.reduce(argmin_reduce_nb, returns_idx=True, to_index=False))
%timeit big_mapped_array.reduce(argmin_reduce_nb, returns_idx=True, to_index=False)

print(mapped_array_grouped.reduce(argmin_reduce_nb, returns_idx=True))
%timeit big_mapped_array_grouped.reduce(argmin_reduce_nb, returns_idx=True)

In [33]:
@njit
def min_max_reduce_nb(col, a):
    return np.array([np.min(a), np.max(a)])

print(mapped_array[(0, 'a')].reduce(min_max_reduce_nb, returns_array=True))
print(mapped_array[[(0, 'a'), (1, 'b')]].reduce(min_max_reduce_nb, returns_array=True))
print(mapped_array.reduce(min_max_reduce_nb, returns_array=True))
print(mapped_array.reduce(min_max_reduce_nb, returns_array=True, wrap_kwargs=dict(name_or_index=['min', 'max'])))
print(mapped_array.reduce(min_max_reduce_nb, returns_array=True, wrap_kwargs=dict(name_or_index=['min', 'max']), fill_value=0.))
print(mapped_array.reduce(min_max_reduce_nb, returns_array=True, wrap_kwargs=dict(to_timedelta=True)))

%timeit big_mapped_array.reduce(min_max_reduce_nb, returns_array=True)

In [34]:
print(mapped_array_grouped[0].reduce(min_max_reduce_nb, returns_array=True))
print(mapped_array_grouped[[0]].reduce(min_max_reduce_nb, returns_array=True))
print(mapped_array_grouped.reduce(min_max_reduce_nb, returns_array=True))
print(mapped_array_grouped.reduce(min_max_reduce_nb, returns_array=True, group_by=False))

%timeit big_mapped_array_grouped.reduce(min_max_reduce_nb, returns_array=True)

In [35]:
@njit
def idxmin_idxmax_reduce_nb(col, a):
    return np.array([np.argmin(a), np.argmax(a)])

print(mapped_array.reduce(idxmin_idxmax_reduce_nb, returns_array=True, returns_idx=True))
%timeit big_mapped_array.reduce(idxmin_idxmax_reduce_nb, returns_array=True, returns_idx=True)

print(mapped_array.reduce(idxmin_idxmax_reduce_nb, returns_array=True, returns_idx=True, to_index=False))
%timeit big_mapped_array.reduce(idxmin_idxmax_reduce_nb, returns_array=True, returns_idx=True, to_index=False)

print(mapped_array_grouped.reduce(idxmin_idxmax_reduce_nb, returns_array=True, returns_idx=True))
%timeit big_mapped_array_grouped.reduce(idxmin_idxmax_reduce_nb, returns_array=True, returns_idx=True)

In [36]:
print(mapped_array.nth(0))
print(mapped_array.nth(-1))
%timeit big_mapped_array.nth(0)

print(mapped_array_grouped.nth(0))
%timeit big_mapped_array_grouped.nth(0)

In [37]:
print(mapped_array.to_pd().vbt.min())
%timeit big_mapped_array.to_pd().vbt.min()

print(mapped_array.min())
%timeit big_mapped_array.min()

print(mapped_array_grouped.min())
%timeit big_mapped_array_grouped.min()

In [38]:
print(mapped_array.to_pd().vbt.max())
%timeit big_mapped_array.to_pd().vbt.max()

print(mapped_array.max())
%timeit big_mapped_array.max()

print(mapped_array_grouped.max())
%timeit big_mapped_array_grouped.max()

In [39]:
print(mapped_array.to_pd().vbt.mean())
%timeit big_mapped_array.to_pd().vbt.mean()

print(mapped_array.mean())
%timeit big_mapped_array.mean()

print(mapped_array_grouped.mean())
%timeit big_mapped_array_grouped.mean()

In [40]:
print(mapped_array.to_pd().vbt.median())
%timeit big_mapped_array.to_pd().vbt.median()

print(mapped_array.median())
%timeit big_mapped_array.median()

print(mapped_array_grouped.median())
%timeit big_mapped_array_grouped.median()

In [41]:
print(mapped_array.to_pd().vbt.std())
print(mapped_array.to_pd().vbt.std(ddof=0))
%timeit big_mapped_array.to_pd().vbt.std()

print(mapped_array.std())
print(mapped_array.std(ddof=0))
%timeit big_mapped_array.std()

print(mapped_array_grouped.std())
%timeit big_mapped_array_grouped.std()

In [42]:
print(mapped_array.to_pd().vbt.sum())
%timeit big_mapped_array.to_pd().vbt.sum()

print(mapped_array.sum())
%timeit big_mapped_array.sum()

print(mapped_array_grouped.sum())
%timeit big_mapped_array_grouped.sum()

In [43]:
print(mapped_array.to_pd().vbt.idxmin())
%timeit big_mapped_array.to_pd().vbt.idxmin()

print(mapped_array.idxmin())
%timeit big_mapped_array.idxmin()

print(mapped_array_grouped.idxmin())
%timeit big_mapped_array_grouped.idxmin()

In [44]:
print(mapped_array.to_pd().vbt.idxmax())
%timeit big_mapped_array.to_pd().vbt.idxmax()

print(mapped_array.idxmax())
%timeit big_mapped_array.idxmax()

print(mapped_array_grouped.idxmax())
%timeit big_mapped_array_grouped.idxmax()

In [45]:
print(mapped_array.to_pd().vbt.describe())
print(mapped_array.to_pd().vbt.describe(percentiles=[0.3, 0.7]))
%timeit big_mapped_array.to_pd().vbt.describe()

print(mapped_array.describe())
print(mapped_array.describe(percentiles=[0.3, 0.7]))
%timeit big_mapped_array.describe()

print(mapped_array_grouped.describe())
%timeit big_mapped_array_grouped.describe()

In [46]:
print(mapped_array.to_pd().vbt.count())
%timeit big_mapped_array.to_pd().vbt.count()

print(mapped_array.count())
%timeit big_mapped_array.count()

print(mapped_array_grouped.count())
%timeit big_mapped_array_grouped.count()

In [47]:
mapping = {x: str(x) for x in np.unique(mapped_array.values)}
big_mapping = {x: str(x) for x in np.unique(big_mapped_array.values)}

In [48]:
print(mapped_array[(0, 'a')].value_counts())
%timeit big_mapped_array[0].value_counts()

print(mapped_array[(0, 'a')].value_counts(mapping=mapping))
%timeit big_mapped_array[0].value_counts(mapping=big_mapping)

print(mapped_array.value_counts())
%timeit big_mapped_array.value_counts()

print(mapped_array.value_counts(mapping=mapping))
%timeit big_mapped_array.value_counts(mapping=big_mapping)

print(mapped_array_grouped.value_counts())
%timeit big_mapped_array_grouped.value_counts()

In [49]:
print(mapped_array[(0, 'a')].stats())
%timeit big_mapped_array[0].stats(silence_warnings=True)

print(mapped_array.stats(column=(0, 'a')))
%timeit big_mapped_array.stats(column=0, silence_warnings=True)

print(mapped_array.stats())
%timeit big_mapped_array.stats(silence_warnings=True)

print(mapped_array.copy(mapping=mapping).stats())
%timeit big_mapped_array.copy(mapping=big_mapping).stats(silence_warnings=True)

In [50]:
mapped_array[(0, 'a')].histplot().show_svg()
mapped_array.histplot().show_svg()
mapped_array_grouped.histplot().show_svg()

In [51]:
mapped_array[(0, 'a')].boxplot().show_svg()
mapped_array.boxplot().show_svg()
mapped_array_grouped.boxplot().show_svg()

In [11]:
mapped_array[(0, 'a')].plots().show_svg()
mapped_array.plots().show_svg()

## Records

In [52]:
print(records[(0, 'a')].values)
print(records[(0, 'a')].wrapper.columns)

print(records[(1, 'b')].values)
print(records[(1, 'b')].wrapper.columns)

print(records[[(0, 'a'), (0, 'a')]].values)
print(records[[(0, 'a'), (0, 'a')]].wrapper.columns)

print(records[[(0, 'a'), (1, 'b')]].values)
print(records[[(0, 'a'), (1, 'b')]].wrapper.columns)

%timeit big_records.iloc[0]
%timeit big_records.iloc[:]

In [53]:
print(records_nosort[(0, 'a')].values)
print(records_nosort[(0, 'a')].wrapper.columns)

print(records_nosort[(1, 'b')].values)
print(records_nosort[(1, 'b')].wrapper.columns)

print(records_nosort[[(0, 'a'), (0, 'a')]].values)
print(records_nosort[[(0, 'a'), (0, 'a')]].wrapper.columns)

print(records_nosort[[(0, 'a'), (1, 'b')]].values)
print(records_nosort[[(0, 'a'), (1, 'b')]].wrapper.columns)

%timeit big_records_nosort.iloc[0]
%timeit big_records_nosort.iloc[:]

In [54]:
print(records_grouped[0].wrapper.columns)  # indexing on groups, not columns!
print(records_grouped[0].wrapper.ndim)
print(records_grouped[0].wrapper.grouped_ndim)
print(records_grouped[0].wrapper.grouper.group_by)

print(records_grouped[1].wrapper.columns)
print(records_grouped[1].wrapper.ndim)
print(records_grouped[1].wrapper.grouped_ndim)
print(records_grouped[1].wrapper.grouper.group_by)

print(records_grouped[[0]].wrapper.columns)
print(records_grouped[[0]].wrapper.ndim)
print(records_grouped[[0]].wrapper.grouped_ndim)
print(records_grouped[[0]].wrapper.grouper.group_by)

print(records_grouped[[0, 1]].wrapper.columns)
print(records_grouped[[0, 1]].wrapper.ndim)
print(records_grouped[[0, 1]].wrapper.grouped_ndim)
print(records_grouped[[0, 1]].wrapper.grouper.group_by)

%timeit big_records_grouped.iloc[0]
%timeit big_records_grouped.iloc[:]

In [55]:
print(records.wrapper.index)
print(records.wrapper.columns)
print(records.wrapper.ndim)
print(records.wrapper.grouper.group_by)

print(records_grouped.wrapper.index)
print(records_grouped.wrapper.columns)
print(records_grouped.wrapper.ndim)
print(records_grouped.wrapper.grouper.group_by)

In [56]:
print(records.values)

print(records.recarray)
%timeit big_records.recarray

In [57]:
print(records.records)

In [58]:
print(records.is_sorted())
%timeit big_records.is_sorted()

print(records_nosort.is_sorted())
%timeit big_records_nosort.is_sorted()

print(records.is_sorted(incl_id=True))
%timeit big_records.is_sorted(incl_id=True)

print(records_nosort.is_sorted(incl_id=True))
%timeit big_records_nosort.is_sorted(incl_id=True)

In [59]:
print(records.sort().records_arr)
%timeit big_records.sort()

print(records_nosort.sort().records_arr)
%timeit big_records_nosort.sort()

print(records.sort(incl_id=True).records_arr)
%timeit big_records.sort(incl_id=True)

print(records_nosort.sort(incl_id=True).records_arr)
%timeit big_records_nosort.sort(incl_id=True)

In [10]:
mask = records.values['some_field1'] >= records.values['some_field1'].mean()
print(records.apply_mask(mask).values)

big_mask = big_records.values['some_field1'] >= big_records.values['some_field1'].mean()
%timeit big_records.apply_mask(big_mask)

In [61]:
@njit
def map_nb(record):
    return record.some_field1 + record.some_field2 * 2

print(records.map(map_nb).sum())
print(records_grouped.map(map_nb).sum())
print(records_grouped.map(map_nb, group_by=False).sum())

%timeit vbt.records.MappedArray(\
    big_wrapper,\
    big_records_arr['some_field1'] + big_records_arr['some_field2'] * 2,\
    big_records_arr['col'],\
)
%timeit big_records.map(map_nb)  # faster

In [62]:
print(records.map_field('col').values)
print(records.map_field('idx').values)
print(records.map_field('some_field1').values)
print(records.map_field('some_field2').values)

print(records.map_field('some_field1').sum())
print(records_grouped.map_field('some_field1').sum())
print(records_grouped.map_field('some_field1', group_by=False).sum())

%timeit big_records.map_field('some_field1')

In [63]:
print(records.map_array(records_arr['some_field1'] + records_arr['some_field2'] * 2).values)
print(records_grouped.map_array(records_arr['some_field1'] + records_arr['some_field2'] * 2).sum())
print(records_grouped.map_array(records_arr['some_field1'] + records_arr['some_field2'] * 2, group_by=False).sum())

%timeit big_records.map_array(big_records_arr['some_field1'] + big_records_arr['some_field2'] * 2)

In [64]:
print(records.count())
print(records_grouped.count())
print(records_grouped.count(group_by=False))

%timeit big_records.count()
%timeit big_records_grouped.count()

In [11]:
filter_mask = np.array([True, False, False, False, False, False, False, False, True])

print(records.apply_mask(filter_mask).count())
print(records_grouped.apply_mask(filter_mask).count())
print(records_grouped.apply_mask(filter_mask, group_by=False).count())

In [12]:
filtered_records = records.apply_mask(filter_mask)
print(filtered_records.records)

In [67]:
print(filtered_records[(0, 'a')].records)
print(filtered_records[(0, 'a')].map_field('some_field1').values)
print(filtered_records[(0, 'a')].map_field('some_field1').min())
print(filtered_records[(0, 'a')].count())

In [68]:
print(filtered_records[(1, 'b')].records)
print(filtered_records[(1, 'b')].map_field('some_field1').values)
print(filtered_records[(1, 'b')].map_field('some_field1').min())
print(filtered_records[(1, 'b')].count())

In [69]:
print(filtered_records[(1, 'c')].records)
print(filtered_records[(1, 'c')].map_field('some_field1').values)
print(filtered_records[(1, 'c')].map_field('some_field1').min())
print(filtered_records[(1, 'c')].count())

In [70]:
print(filtered_records[(1, 'd')].records)
print(filtered_records[(1, 'd')].map_field('some_field1').values)
print(filtered_records[(1, 'd')].map_field('some_field1').min())
print(filtered_records[(1, 'd')].count())

In [12]:
print(records[(0, 'a')].stats())
%timeit big_records[0].stats(silence_warnings=True)

print(records.stats(column=(0, 'a')))
%timeit big_records.stats(column=0, silence_warnings=True)

print(records.stats())
%timeit big_records.stats(silence_warnings=True)

In [13]:
records[(0, 'a')].plots()

## Drawdowns

In [14]:
ts = pd.DataFrame({
    'a': [2, 1, 3, 1, 4, 1], 
    'b': [1, 2, 1, 3, 1, 4],
    'c': [1, 2, 3, 2, 1, 2],
    'd': [1, 2, 3, 4, 5, 6]
}, index=[
    datetime(2020, 1, 1),
    datetime(2020, 1, 2),
    datetime(2020, 1, 3),
    datetime(2020, 1, 4),
    datetime(2020, 1, 5),
    datetime(2020, 1, 6)
])

np.random.seed(42)
big_ts = pd.DataFrame(np.random.randint(1, 10, size=(1000, 1000)))

In [15]:
drawdowns = vbt.Drawdowns.from_ts(ts, freq='1 days')
print(drawdowns.values.shape)

In [16]:
big_drawdowns = vbt.Drawdowns.from_ts(big_ts, freq='1 days')
print(big_drawdowns.values.shape)

In [74]:
%timeit vbt.Drawdowns.from_ts(big_ts, freq='1 days')

In [75]:
print(drawdowns.records)
print(drawdowns.ts)

print(drawdowns['a'].records)
print(drawdowns['a'].ts)

%timeit big_drawdowns.iloc[0]
%timeit big_drawdowns.iloc[:]

In [76]:
print(drawdowns.records_readable)

In [77]:
print(drawdowns['a'].count())

print(drawdowns.count())
print(drawdowns.count(group_by=group_by))

In [80]:
print(drawdowns['a'].drawdown.to_pd())
print(drawdowns.drawdown.to_pd())

%timeit big_drawdowns.drawdown

In [81]:
print(drawdowns['a'].avg_drawdown())
print(drawdowns.avg_drawdown())
%timeit big_drawdowns.avg_drawdown()

print(drawdowns.avg_drawdown(group_by=group_by))
%timeit big_drawdowns.avg_drawdown(group_by=big_group_by)

In [82]:
print(drawdowns['a'].max_drawdown())
print(drawdowns.max_drawdown())
%timeit big_drawdowns.max_drawdown()

print(drawdowns.max_drawdown(group_by=group_by))
%timeit big_drawdowns.max_drawdown(group_by=big_group_by)

In [83]:
print(drawdowns['a'].recovery_return.to_pd())
print(drawdowns.recovery_return.to_pd())

%timeit big_drawdowns.recovery_return

In [84]:
print(drawdowns['a'].avg_recovery_return())
print(drawdowns.avg_recovery_return())
%timeit big_drawdowns.avg_recovery_return()

print(drawdowns.avg_recovery_return(group_by=group_by))
%timeit big_drawdowns.avg_recovery_return(group_by=big_group_by)

In [85]:
print(drawdowns['a'].max_recovery_return())
print(drawdowns.max_recovery_return())
%timeit big_drawdowns.max_recovery_return()

print(drawdowns.max_recovery_return(group_by=group_by))
%timeit big_drawdowns.max_recovery_return(group_by=big_group_by)

In [86]:
print(drawdowns['a'].duration.to_pd())
print(drawdowns.duration.to_pd())

%timeit big_drawdowns.duration

In [87]:
print(drawdowns['a'].avg_duration())
print(drawdowns.avg_duration())
%timeit big_drawdowns.avg_duration()

print(drawdowns.avg_duration(group_by=group_by))
%timeit big_drawdowns.avg_duration(group_by=big_group_by)

In [88]:
print(drawdowns['a'].max_duration())
print(drawdowns.max_duration())
%timeit big_drawdowns.max_duration()

print(drawdowns.max_duration(group_by=group_by))
%timeit big_drawdowns.max_duration(group_by=big_group_by)

In [89]:
print(drawdowns['a'].coverage())
print(drawdowns.coverage())
%timeit big_drawdowns.coverage()

print(drawdowns.coverage(group_by=group_by))
%timeit big_drawdowns.coverage(group_by=big_group_by)

In [90]:
print(drawdowns['a'].decline_duration.to_pd())
print(drawdowns.decline_duration.to_pd())

%timeit big_drawdowns.decline_duration

In [91]:
print(drawdowns['a'].recovery_duration.to_pd())
print(drawdowns.recovery_duration.to_pd())

%timeit big_drawdowns.recovery_duration

In [92]:
print(drawdowns['a'].recovery_duration_ratio.to_pd())
print(drawdowns.recovery_duration_ratio.to_pd())

%timeit big_drawdowns.recovery_duration_ratio

In [17]:
print(drawdowns.active)
print(drawdowns['a'].active.records)
print(drawdowns.active['a'].records)
print(drawdowns.active.records)

%timeit big_drawdowns.active

In [18]:
print(drawdowns.recovered)
print(drawdowns['a'].recovered.records)
print(drawdowns.recovered['a'].records)
print(drawdowns.recovered.records)

%timeit big_drawdowns.recovered

In [95]:
print(drawdowns['a'].active_drawdown())
print(drawdowns.active_drawdown())

%timeit big_drawdowns.active_drawdown()

In [96]:
print(drawdowns['a'].active_duration())
print(drawdowns.active_duration())

%timeit big_drawdowns.active_duration()

In [97]:
print(drawdowns['a'].active_recovery_return())
print(drawdowns.active_recovery_return())

%timeit big_drawdowns.active_recovery_return()

In [98]:
print(drawdowns['a'].active_recovery_duration())
print(drawdowns.active_recovery_duration())

%timeit big_drawdowns.active_recovery_duration()

In [19]:
print(drawdowns['a'].stats())
%timeit big_drawdowns[0].stats(silence_warnings=True)

print(drawdowns.stats(column='a'))
%timeit big_drawdowns.stats(column=0, silence_warnings=True)

print(drawdowns.stats())
%timeit big_drawdowns.stats(silence_warnings=True)

In [12]:
drawdowns['a'].plot().show_svg()

In [13]:
drawdowns.plot(column='a', top_n=1).show_svg()

In [19]:
drawdowns['a'].plots().show_svg()

## Orders

In [20]:
close = pd.Series([1, 2, 3, 4, 5, 6, 7, 8], index=[
    datetime(2020, 1, 1),
    datetime(2020, 1, 2),
    datetime(2020, 1, 3),
    datetime(2020, 1, 4),
    datetime(2020, 1, 5),
    datetime(2020, 1, 6),
    datetime(2020, 1, 7),
    datetime(2020, 1, 8)
]).vbt.tile(4, keys=['a', 'b', 'c', 'd'])
print(close)

big_close = pd.DataFrame(np.random.uniform(1, 10, size=(1000, 1000)))

In [21]:
from vectorbt.portfolio.enums import order_dt

records_arr = np.asarray([
    (0, 0, 0, 1. , 1., 0.01 , 0), (1, 0, 1, 0.1, 2., 0.002, 0),
    (2, 0, 2, 1. , 3., 0.03 , 1), (3, 0, 3, 0.1, 4., 0.004, 1),
    (4, 0, 5, 1. , 6., 0.06 , 0), (5, 0, 6, 1. , 7., 0.07 , 1),
    (6, 0, 7, 2. , 8., 0.16 , 0), (7, 1, 0, 1. , 1., 0.01 , 1),
    (8, 1, 1, 0.1, 2., 0.002, 1), (9, 1, 2, 1. , 3., 0.03 , 0),
    (10, 1, 3, 0.1, 4., 0.004, 0), (11, 1, 5, 1. , 6., 0.06 , 1),
    (12, 1, 6, 1. , 7., 0.07 , 0), (13, 1, 7, 2. , 8., 0.16 , 1),
    (14, 2, 0, 1. , 1., 0.01 , 0), (15, 2, 1, 0.1, 2., 0.002, 0),
    (16, 2, 2, 1. , 3., 0.03 , 1), (17, 2, 3, 0.1, 4., 0.004, 1),
    (18, 2, 5, 1. , 6., 0.06 , 0), (19, 2, 6, 2. , 7., 0.14 , 1),
    (20, 2, 7, 2. , 8., 0.16 , 0)
], dtype=order_dt)
print(records_arr.shape)

wrapper = vbt.ArrayWrapper.from_obj(close, freq='1 days')
orders = vbt.Orders(wrapper, records_arr, close)
orders_grouped = vbt.Orders(wrapper.regroup(group_by), records_arr, close)

In [22]:
big_records_arr = np.asarray(list(zip(*(
    np.arange(1000000),
    np.tile(np.arange(1000), 1000),
    np.repeat(np.arange(1000), 1000),
    np.full(1000000, 10),
    np.random.uniform(1, 10, size=1000000),
    np.full(1000000, 1),
    np.full(1000000, 1)
))), dtype=order_dt)
big_records_arr['side'][::2] = 0
print(big_records_arr.shape)

big_wrapper = vbt.ArrayWrapper.from_obj(big_close, freq='1 days')
big_orders = vbt.Orders(big_wrapper, big_records_arr, big_close)
big_orders_grouped = vbt.Orders(big_wrapper.copy(group_by=big_group_by), big_records_arr, big_close)

In [106]:
print(orders.records)
print(orders.close)

print(orders['a'].records)
print(orders['a'].close)

%timeit big_orders.iloc[0]
%timeit big_orders.iloc[:]

%timeit big_orders_grouped.iloc[0]
%timeit big_orders_grouped.iloc[:]

In [107]:
print(orders.records_readable)

In [23]:
print(orders.buy)
print(orders['a'].buy.records)
print(orders.buy['a'].records)
print(orders.buy.records)

%timeit big_orders.buy

In [24]:
print(orders.sell)
print(orders['a'].sell.records)
print(orders.sell['a'].records)
print(orders.sell.records)

%timeit big_orders.sell

In [25]:
print(orders['a'].stats())
%timeit big_orders[0].stats(silence_warnings=True)

print(orders.stats(column='a'))
%timeit big_orders.stats(column=0, silence_warnings=True)

print(orders.stats())
%timeit big_orders.stats(silence_warnings=True)

In [27]:
orders['a'].plot().show_svg()

In [28]:
orders.plot(column='a').show_svg()

In [23]:
orders.plots(column='a').show_svg()

## Trades

In [24]:
trades = vbt.ExitTrades.from_orders(orders)
trades_grouped = vbt.ExitTrades.from_orders(orders_grouped)
print(trades.values.shape)

In [25]:
big_trades = vbt.ExitTrades.from_orders(big_orders)
big_trades_grouped = vbt.ExitTrades.from_orders(big_orders_grouped)
print(big_trades.values.shape)

In [22]:
%timeit vbt.ExitTrades.from_orders(big_orders)

In [32]:
print(trades.records)
print(trades.close)

print(trades['a'].records)
print(trades['a'].close)

%timeit big_trades.iloc[0]
%timeit big_trades.iloc[:]

%timeit big_trades_grouped.iloc[0]
%timeit big_trades_grouped.iloc[:]

In [33]:
print(trades.records_readable)

In [34]:
print(trades['a'].count())
print(trades.count())
%timeit big_trades.count()

print(trades_grouped.count())
%timeit big_trades_grouped.count()

In [35]:
print(trades.winning)
print(trades['a'].winning.records)
print(trades.winning['a'].records)
print(trades.winning.records)

%timeit big_trades.winning

In [36]:
print(trades.losing)
print(trades['a'].losing.records)
print(trades.losing['a'].records)
print(trades.losing.records)

%timeit big_trades.losing

In [39]:
print(trades['a'].winning_streak.to_pd(ignore_index=True))
print(trades.winning_streak.to_pd(ignore_index=True))

%timeit big_trades.winning_streak

In [41]:
print(trades['a'].losing_streak.to_pd(ignore_index=True))
print(trades.losing_streak.to_pd(ignore_index=True))

%timeit big_trades.losing_streak

In [37]:
print(trades['a'].win_rate())
print(trades.win_rate())
%timeit big_trades.win_rate()

print(trades.win_rate(group_by=group_by))
%timeit big_trades.win_rate(group_by=big_group_by)

In [42]:
print(trades['a'].profit_factor())
print(trades.profit_factor())
%timeit big_trades.profit_factor()

print(trades_grouped.profit_factor())
%timeit big_trades_grouped.profit_factor()

In [43]:
print(trades['a'].expectancy())
print(trades.expectancy())
%timeit big_trades.expectancy()

print(trades_grouped.expectancy())
%timeit big_trades_grouped.expectancy()

In [44]:
print(trades['a'].sqn())
print(trades.sqn())
%timeit big_trades.sqn()

print(trades_grouped.sqn())
%timeit big_trades_grouped.sqn()

In [45]:
print(trades.long)
print(trades['a'].long.records)
print(trades.long['a'].records)
print(trades.long.records)

%timeit big_trades.long

In [46]:
print(trades.short)
print(trades['a'].short.records)
print(trades.short['a'].records)
print(trades.short.records)

%timeit big_trades.short

In [47]:
print(trades.open)
print(trades['a'].open.records)
print(trades.open['a'].records)
print(trades.open.records)

%timeit big_trades.open

In [48]:
print(trades.closed)
print(trades['a'].closed.records)
print(trades.closed['a'].records)
print(trades.closed.records)

%timeit big_trades.closed

In [49]:
print(trades['a'].stats())
%timeit big_trades[0].stats(silence_warnings=True)

print(trades.stats(column='a'))
%timeit big_trades.stats(column=0, silence_warnings=True)

print(trades.stats())
%timeit big_trades.stats(silence_warnings=True)

In [27]:
trades['c'].plot().show_svg()

In [28]:
trades['c'].plot_pnl().show_svg()

In [29]:
trades.plots(column='c').show_svg()

## Positions

In [ ]:
positions = vbt.Positions.from_trades(trades)
positions_grouped = vbt.Positions.from_trades(trades_grouped)
print(positions.values.shape)

In [31]:
big_positions = vbt.Positions.from_trades(big_trades)
big_positions_grouped = vbt.Positions.from_trades(big_trades_grouped)
print(big_positions.values.shape)

In [31]:
%timeit vbt.Positions.from_trades(big_trades)

In [55]:
print(positions.records)
print(positions.close)

print(positions['a'].records)
print(positions['a'].close)

%timeit big_positions.iloc[0]
%timeit big_positions.iloc[:]

%timeit big_positions_grouped.iloc[0]
%timeit big_positions_grouped.iloc[:]